In [ ]:
pip install tensorflow
pip install tensor-estimator
# pip install --upgrade tensorflow
# pip install --upgrade tensor-estimator
pip install pandas
pip install scikit-learn
pip install pandas scikit-learn tensorflow

In [1]:
import tensorflow as tf
import tensorflow_estimator as tf_estimator
import pandas as pd


from sklearn.model_selection import train_test_split

In [2]:
# The SyntaxError: (unicode error) occurs because backslashes (\) in file paths are interpreted as escape characters in Python strings. 
# To avoid this issue, you can use raw strings by prefixing the string with r, or replace backslashes with forward slashes (/),
# or double the backslashes (\\).

census = pd.read_csv(r"C:\Users\phoss\OneDrive\Documents\Dataset\Tensorflow\What is Tensorflow\census_data_What is Tensorflow.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
#Convert the label column to 0s and 1s instead of string as thats our target column
census['income_bracket'].unique()

def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1
    
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [5]:
#perform a train-test split on the data
x_data = census.drop('income_bracket', axis=1)
y_labels = census['income_bracket']

X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.3, random_state=101) 

In [6]:
#create the feature columns for the categorial values using vocabulary lists or has buckets
#Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use has buckets.

gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female","Male"])
occupations = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=10000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=10000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=10000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=10000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=10000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=10000)

In [7]:
#Create the continous feature_coluns for the continous values using numeric_column

age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [8]:
#Put all these variables into a single list with the variable name feat_cols

feat_cols = [gender, occupations, marital_status, relationship, education, workclass, native_country, age, education_num, capital_gain, capital_loss, hours_per_week]

In [9]:
# Create the input function.
# input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, bactch_size=100, num_epochs=None, shuffle=True)

# Define input functions using tf.data.Dataset
def input_fn(features, labels, training=True, batch_size=100):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset = dataset.shuffle(buffer_size=len(features))
    dataset = dataset.batch(batch_size)
    return dataset

In [10]:
input_func = lambda: input_fn(X_train, y_train)
#test_input_func = lambda: input_fn(X_test, y_test, training=False)

In [11]:
# Create the model with tf.estimator using LinerClassifier
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\phoss\\AppData\\Local\\Temp\\tmpo3yovoba', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [28]:
model.train(input_fn=input_func, steps=5000)

INFO:tensorflow:Calling model_fn.


C:\Users\phoss\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\phoss\AppData\Local\Temp\tmpo3yovoba\model.ckpt-456
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 456...
INFO:tensorflow:Saving checkpoints for 456 into C:\Users\phoss\AppData\Local\Temp\tmpo3yovoba\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 456...
INFO:tensorflow:loss = 1.1522338, step = 456
INFO:tensorflow:global_step/sec: 146.384
INFO:tensorflow:loss = 0.5875862, step = 556 (0.699 sec)
INFO:tensorflow:global_step/sec: 330.175
INFO:tensorflow:loss = 0.39940506, step = 656 (0.287 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 684...
INFO:tensorflow:Saving checkpoints for 684 into C:\Users\phoss\AppData\Local\Temp\tmpo3yovoba\model.ckpt.
INFO:tenso

In [29]:
# pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=len(X_test), shuffle=False)

# Define input function for prediction using tf.data.Dataset
def predict_input_fn(features, batch_size=100):
    dataset = tf.data.Dataset.from_tensor_slices(dict(features))
    dataset = dataset.batch(batch_size)
    return dataset

# Example usage for prediction
pred_fn = lambda: predict_input_fn(features=X_test, batch_size=len(X_test))

In [30]:
#use model.predit() and pass in your input function. THis will produce a generator of predictions.
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\phoss\AppData\Local\Temp\tmpo3yovoba\model.ckpt-684
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [31]:
predictions[0]

{'logits': array([-2.0810642], dtype=float32),
 'logistic': array([0.11095092], dtype=float32),
 'probabilities': array([0.88904905, 0.11095095], dtype=float32),
 'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'all_class_ids': array([0, 1]),
 'all_classes': array([b'0', b'1'], dtype=object)}

In [32]:
#Create a list of class_ids key values from the predition list of dictionaries. These predictions will be used to compare against y_test values
final_preds=[]
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [33]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [34]:
#calculate model performance on Test Data
from sklearn.metrics import classification_report

In [35]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.86      0.91      0.88      7436
           1       0.65      0.52      0.58      2333

    accuracy                           0.82      9769
   macro avg       0.76      0.72      0.73      9769
weighted avg       0.81      0.82      0.81      9769

